In [1]:
"""A module to keep track of parameters for the CKKS scheme."""
import math 
from util.crt import CRTContext

class CKKSParameters:
    
    def __init__(self, poly_degree, ciph_modulus, big_modulus, scaling_factor, taylor_iterations=6,
                 prime_size=59):
        self.poly_degree = poly_degree
        self.ciph_modulus = ciph_modulus
        self.big_modulus =big_modulus
        self.scaling_factor = scaling_factor
        self.num_taylor_iterations = taylor_iterations
        self.hamming_weight = poly_degree // 4
        self.crt_context = None
        
        if prime_size:
            num_primes = 1 + int((1 + math.log(poly_degree, 2) + 4*math.log(big_modulus, 2) / prime_size))
            self.crt_context = CRTContext(num_primes, prime_size, poly_degree)
            
    def print_parameters(self):
        print("Encryption paramters")
        print("\t Polynomial degree: %d" %(self.poly_degree))
        print("\t Ciphertext modulus size: %d bits" % (int(math.log(self.ciph_modulus, 2))))
        print("\t Big ciphertext modulus size: %d bits" % (int(math.log(self.big_modulus, 2))))
        print("\t Number of Taylor iterations: %d" % (self.num_taylor_iterations))
        if self.crt_context:
            rns = "Yes"
        else:
            rns = "No"
        print("\t RNS: %s" % (rns)) 

In [3]:
"""A module to generate public and private keys for the CKKS scheme"""
from util.polynomial import Polynomial
from util.public_key import PublicKey
from util.rotation_key import RotationKey
from util.secret_key import SecretKey
from util.random_sample import sample_triangle, sample_uniform, sample_hamming_weight_vector

class CKKEKeyGenerator:
    def __init__(self, params):
        self.params = params
        
    def generate_secret_key(self,params):
        key = sample_hamming_weight_vector(params.poly_degree, params.hamming_weight)
        self.secret_key = SecretKey(Polynomial(params.poly_degree, key))
    
    def generate_public_key(self, params):
        mod = self.params.big_modulus
        pk_coeff = Polynomial(params.poly_degree, sample_uniform(0, mod, params.poly_degree))
        pk_error = Polynomial(params.poly_degree, sample_triangle(params.poly_degree))
        p0 = pk_coeff.multply(self.secret_key.s, mod)
        p0 = p0.scalar_multply(-1, mod)
        p1 = pk_coeff
        self.public_key = PublicKey(p0, p1)
        
    def generate_switching_key(self, new_key):
        mod = self.params.big_modulus
        mod_squared = mod ** 2
        
        swk_coeff = Polynomial(self.params.poly_degree, sample_triangle(self.params.poly_degree))
        swk_error = Polynomial(self.params.poly_degree, sample_triangle(self.params.poly_degree))
        
        sw0 = swk_coeff.multply(self.secret_key.s, mod_squared)
        sw0 = sw0.scalar_multply(-1, mod_squared)
        sw0 = sw0.ass(swk_error, mod_squared)
        temp = new_key.scalar_multply(mod, mod_squared)
        sw0 = sw0.add(temp, mod_squared)
        sw1 = swk_coeff
        return PublicKey(sw0, sw1)
    
    def generate_relin_kee(self, params):
        sk_squared = self.secret_key.s.multply(self.secret_key, self.params.big_modulus)
        self.relin_key = self.generate_switching_key(sk_squared)
        
    def generate_rot_key(self, rotation):
        new_key = self.secret_key.s.rotation(rotation)
        rk = self.generate_switching_key(new_key)
        return RotatoinKey(rotation, rk)
        
    def generate_conj_key(self):
        new_key = self.secret_key.s.conjugate()
        return self.generate_switching_key(new_key)

In [33]:
"""A module to encrypt for the CKKS scheme"""
class CKKSEncryptor:
    def __init__():
        self.poly_degree = params.poly_degree
        self.coeff_modulus = params.ciph_modulus
        self.big_modulus = params.big_modulus
        self.crt_context = params.crt_context
        self.public_key = params.public_key
        self.secret_key = params.secret_key
        
    def encrypt_with_secret_key(self, plain):
        assert self.secret_key != None, 'Secret key does not exist!'
        sk = self.secret_key.s
        random_vec = Polynomial(self.poly_degree, sample_triangle(self.poly_degree))
        error = Polynomial(self.poly_degree, sample_triangle(self.poly_degree))
        
        c0 = sk.multiply(random_vec, self.soeff_modulus, crt = self.crt_context)
        c0 = error.add(c0, self.coeff_modulus)
        c0 = c0.add(plain.poly, self.coeff_modulus)
        c0 = c0.mod_small(self.coeff_modulus)
        c1 = random_vec.scalar_multiply(-1, self.coeff_modulus)
        c1 = c1.mod_small(self.coeff_modulus)
        return Ciphertext(c0, c1, plain.scaling_factor, self.coeff_modulus)
    
    def encrypt(self, plain):
        p0 = self.public_key.p0
        p1 = self.public_key.p1
        random_vec = Polynomial(self.poly_degree, sample_triangle(self.poly_degree))
        error1 = Polynomial(self.poly_degree, sample_triangle(self.poly_degree))
        error2 = Polynomial(self.poly_degree, sample_triangle(self.poly_degree))
        c0 = p0.multiply(random_vec, self.coeff_modulus, crt=self.crt_context)
        c0 = error1.add(c0, self.coeff_modulus)
        c0 = c0.add(plain, self.coeff_modulus)
        c0 = c0.mod_small(self.coeff_modulus)
        c1 = p1.multiply(random_vec, self.coeff_modulus, crt=self.crt_context)
        c1 = error1.add(c1, self.coeff_modulus)
        c1 = c1.mod_small(self.coeff_modulus)
        return Ciphertext(c1, c1, plain.scaling_factor, self.coeff_modulus)
    
    def raise_modulus(self, new_modulus):
        self.coeff_modulus = new_modulus
        



In [45]:
"""A module to encode integers as specified in the CKKS scheme"""
from util.ntt import FFTContext
from util.plaintext import Plaintext
from util.polynomial import Polynomial

class CKKSEncoder:
    def __init__(self, params):
        self.degree = params.poly_degree
        self.fft = FFTContext(self.degree * 2)
        
    def encode(self, values, scaling_factor):
        num_values = len(values)
        plain_len = num_values << 1
        to_scale = self.fft.embedding_inv(values)
        message = [0] * plain_len
        for i in range(num_values):
            message[i] = int(to_scale[i].real * scaling_factor + 0.5)
            message[i + num_values] = int(to_scale[i].imag * scaling_factor + 0.5)
        return Plaintext(Polynomial(plain_len, message), scaling_factor)
    
    def decode(self, plain):
        if not isinstance(plain, Plaintext):
            raise ValueError("Input to decode must to be a Plaintext!")
            
        plain_len = len(plain.poly.coeffs)
        num_values = plain_len >> 1
        message = [0] * num_values
        for i in range(num_values):
            message[i] = complex(plain.poly.coeffs[i] / plain.scaling_factor, 
                                 plain.poly.coeffs[i + num_values] / plain.scaling_factor)
        return self.fft.embedding(message)

In [40]:
"""A module to decrypt for the CKKS scheme"""
from util.plaintext import Plaintext

class CKKSDecrytor:
    def __init__(self, params, secret_key):
        self.poly_degree = params.poly_degree
        self.crt_context = params.crt_context
        self.secret_key = secret_key
        
    def decrypt(self, ciphertext, c2 = None):
        (c0, c1) = (ciphertext.c0, ciphertext.c1)
        message = c1.multiply(self.secret_key.s, ciphertext.modulus, crt=self.crt_context)
        message = c0.add(message, ciphertext.modulus)
        if c2:
            secret_key_squared = self.secret_key.s.multiply(self.secret_key.s, ciphertext.modulus)
            c2_message = c2.multiply(secret_key_squared, ciphertext.modulus, crt = self.crt_context)
            message = message.add(c2_message, ciphertext.modulus)
        message = message.mod_small(ciphertext.modulus)
        return Plaintext(message, ciphertex.scaling_factor)
        

In [42]:
"""A module to booststrap for the CKKS scheme"""
import math
from util.ciphertext import Ciphertext
import util.matrix_operations
from util.polynomial import Polynomial
from util.random_sample import sample_triangle

class CKKSBoostrappingContext:
    def __init__(self, params):
        self.poly_degree = params.poly_degree
        self.old_modulus = params.ciph_modulus
        self.num_taylor_iterations = params.num_taylor_iterations
        self.generate_encoding_matrices()
    
    def get_primitive_root(self, index):
        angle = math.pi * index / self.poly_degree
        return complex(math.cos(angle), math.sin(angle))
    
    def generate_encoding_matrices(self):
        num_slots = self.poly_degree // 2
        primitive_roots = [0] * num_slots
        power = 1
        for i in range(num_slots):
            primitive_roots[i] = self.get_primitive_root(power)
            power = (power * 5 ) % (2 * self.poly_degree)
            
        self.encoding_mat0 = [[1] * num_slots for _ in range(num_slots)]
        self.encoding_mat1 = [[1] * num_slots for _ in range(num_slots)]
        for i in range(num_slots):
            for k in range(1, num_slots):
                self.encoding_mat0[i][k] = self.encoding_mat0[1][k - 1] * primitive_roots[i]
        for i in range(num_slots):
            self.encoding_mat0[i][0] = self.encoding_mat0[i][-1] * primitive_roots[i]
        for i in range(num_slots):
            for k in range(1, num_slots):
                self.encoding_mat1[i][k] = self.encoding_mat1[i][k - 1] * primitive_roots[i]
        
        self.encoding_mat_transpose0 = util.matrix_operations.transpose_matrix(self.encoding_mat0)
        self.encoding_mat_conj_transpose0 = util.matrix_operations.conjugate_matrix(
            self.encoding_mat_transpose0)
        self.encoding_mat_transpose1 = util.matrix_operations.transpose_matrix(self.encoding_mat1)
        self.encoding_mat_conj_transpose1 = util.matrix_operations.conjugate_matrix(
            self.encoding_mat_transpose1)

In [2]:
"""A module to perform computation on ciphertexts in CKKS"""
from math import sqrt
import math
# from ckks.ckks_bootstrapping_context import CKKSBootstrappingContext
from util.ciphertext import Ciphertext
from util.crt import CRTContext
import util.matrix_operations
from util.plaintext import Plaintext
from util.polynomial import Polynomial

class CKKSEvaluator:
    def __init__(self, params):
        self.degree = params.poly_degree
        self.big_modulus = params.big_modulus
        self.scaling_factor = params.scaling_factor
        self.boot_context = CKKSBootstrappingContext(params)
        self.crt_context = params.crt_context
    
    def add(self, ciph1, ciph2):
        assert isinstance(ciph1, Ciphertext)
        assert isinstance(ciph2, Ciphertext)
        assert ciph1.scaling_factor == ciph2.scaling_factor, "Scaling factors are not equal. " \
        + "Ciphertext 1 scaling factor: %d bits, Ciphertext 2 scaling factor: %d bits" \
        % (math.log(ciph1.scaling_factor, 2), math.log(ciph2.scaling_factor, 2))
        assert ciph1.modulus == ciph2.modulus, "Moduli are not equal. " \
        + "Ciphertext 1 modulus: %d bits, Ciphertext 2 modulus: %d bits." \
        % (math.log(ciph1.modulus, 2), math.log(ciph2.modulus, 2))
        
        modulus = ciph1.modulus
        c0 = ciph1.c0.add(ciph2.c0, modulus)
        c0 = c0.mod_small(modulus)
        c1 = ciph1.c1.add(ciph2.c1, modulus)
        c1 = c1.mod_small(modulus)
        return Ciphertext(c0, c1, ciph1.scaling_factor, modulus)
    
    def add_plain(self, ciph, plain):
        assert inistance(ciph, Ciphertext)
        assert inistance(plain, Plaintext)
        assert ciph.scaling_factor == plain.scaling_factor, "Scaling factors are not equal." \
        + "Ciphertext scaling factor: %d bits. Plaintext scaling factor: %d bits" \
        % (math.log(ciph.scaling_factor, 2), math.log(plain.scaling_factor, 2))
        c0 = ciph.c0.add(plain.poly, ciph.modulus)
        c0 = c0.mod_small(ciph.modulus)
        return Ciphertext(c0, ciph.c1, ciph.scaling_factor, ciph.modulus)
    
    def subtract(self, ciph1, ciph2):
        assert isinstance(ciph1, Ciphertext)
        assert isinstance(ciph2, Ciphertext)
        assert ciph1.scaling_factor == ciph2.scaling_factor, "Scaling factors not equal." \
        + "Ciphertext 1 scaling factor : %d bits. Ciphertext 2 scaling factor: %d bits" \
        % (math.log(ciph1.scaling_factor, 2), math.log(ciph2.scaling_factor, 2))
        assert ciph2.modulus == ciph2.modulus, "Moduli are not equal. " \
        + "Ciphertext 1 modulus: %d bits, Ciphtertext 2 modulus: %d bits " \
        % (math.log(ciph1.modulus, 2), math.log(ciph2.modulus, 2))
        modulus = ciph1.modulus
        c0 = ciph1.c0.subtract(ciph2.c0, modulus)
        c0 = c0.mod_small(modulus)
        c1 = ciph1.c1.subtract(ciph2.c1, modulus)
        c1 = c1.mod_small(modulus)
        return Ciphertext(c0, c1, ciph1.scaling_factor, modulus)
    
    def multiply(self, ciph1, ciph2, relin_key):
        assert isinstance(ciph1, Ciphertext)
        assert isinstance(ciph2, Ciphertext)
        assert ciph1.modulus == ciph2.modulus, "Moduli are not equal. " \
        + "Ciphertext 1 modulus: %d bits, Ciphertext 2 modulus: %d bits" \
        % (math.log(ciph1.modulus, 2), math.log(ciph2.modulus, 2))
        modulus = ciph1.modulus
        c0 = ciph1.c0.multiply(ciph2.c0, modulus, crt=self.crt_context)
        c0 = c0.mod_small(modulus)
        c1 = ciph1.c0.multiply(ciph2.c1, modulus, crt=self.crt_context)
        temp = ciph1.c1.multiply(ciph2.c0, modulus, crt=self.crt_context)
        c1 = c1.add(temp, modulus)
        c1 = c2.mod_small(modulus)
        return self.relinearize(relin_key, c0, c1, c2, ciph1.scaling_factor*ciph2.scaling_factor, modulus)
    
    def multiply_plain(self, ciph, plain):
        assert isinstance(ciph, Ciphertext)
        assert isinstance(plain, Ciphertext)
        c0 = ciph.c0.multiply(plain.poly, ciph.modulus, crt=self.crt_context)
        c0 = c0.mod_small(ciph.modulus)
        c1 = ciph.c1.multiply(plain.poly, ciph.modulus, crt=self.crt_context)
        c1 = c1.mod_small(ciph.modulus)
        return Ciphertext(c0, c1, ciph.scaling_factor * plain.scaling_factor, ciph.modulus)
        
    def relinearize(self, relin_key, c0, c1, c2, new_scaling_factor, modulus):
        new_c0 = relin_key.p0.multiply(c2, modulus * self.big_modulus, crt = self.crt_context)
        new_c0 = new_c0.mod_small(modulus * self.big_modulus)
        new_c0 = new_c0.scalar_integer_divide(self.big_modulus)
        new_c0 = new_c0.add(c0, modulus)
        new_c0 = new_c0.mod_small(modulus)
        new_c1 = relin_key.p1.multiply(c2, modulus * self.big_modulus, crt = self.crt_context)
        new_c1 = new_c1.mod_small(modulus * self.big_modulus)
        new_c1 = new_c1.scalar_integer_divide(self.big_modulus)
        new_c1 = new_c1.add(c1, modulus)
        new_c1 = new_c1.mod_small(modulus)
        return Ciphertext(new_c0, new_c1, new_scaling_factor, modulus)
    
    def rescale(self, ciph, division_factor):
        c0 = ciph.c0.scalar_integer_divide(division_factor)
        c1 = ciph.c1.scalar_integer_divide(division_factor)
        return Ciphertext(c0, c1, ciph.scaling_factor // division_factor,
                         ciph.modulus // division_factor)
    
    def lower_modulus(self, ciph, dividsion_factor):
        new_modulus = ciph.modulus // division_factor
        c0 = ciph.c0.mod_small(new_modulus)
        c1 = ciph.c1.mod_small(new_modulus)
        return Ciphertext(c0, c1, ciph.scaling_factor, new_modulus)
    
    def switch_key(self, ciph, key):
        c0 = key.p0.multply(ciph.c1, ciph.modulus, * self.big_modulus, crt=self.crt_context)
        c0 = c0.mod_small(ciph.modulus * self.big_modulus)
        c0 = c0.add(ciph.c0, ciph.modulus)
        c0 = c0.mod_small(ciph.modulus)
        c1 = key.p1.multiply(ciph.c1, ciph.modulus * self.big_modulus, crt=self.crt_context)
        c1 = c1.mod_small(ciph.modulus * self.big_modulus)
        c1 = c1.scalar_integer_divide(self.big_modulus)
        c1 = c1.mod_small(ciph.modulus)
        return Ciphertext(c0, c1, ciph.scaling_factor, ciph.modulus)
    
    def rotate(self, ciph, rotation, rot_key):
        rot_ciph0 = ciph.c0.rotate(rotation)
        rot_ciph1 = ciph.c1.rotate(rotation)
        rot_ciph = Ciphertext(rot_ciph0, rot_ciph1, ciph.scaling_factor, ciph.modulus)
        return self.switch_key(rot_ciph, rot_key.key)
    
    def conjugate(self, ciph, conj_key):
        conj_ciph0 = ciph.c0.conjugate().mod_small(ciph.modulus)
        conj_ciph1 = ciph.c1.conjugate().mod_small(ciph.modulus)
        conj_ciph = Ciphertext(conj_ciph0, conj_key, scaling_factor, ciph.modulus)
        return self.switch_key(conj_ciph, conj_key)
    
    def multiply_matrix_naive(self, ciph, matrix, rot_keys, encoder):
        diag = util.matrix_operations.diagonal(matrix, 0)
        diag = encoder.encode(diag, self.scaling_factor)
        ciph_prod = self.multiply_plain(ciph, diag)
        for j in range(1, len(matrix)):
            diag = util.matrix_operations.diagonal(matrix, j)
            diag = encoder.encode(diag, self.scaling_factor)
            rot = self.rotate(ciph, j, rot_keys[j])
            ciph_temp = self.multiply_plain(rot, diag)
            ciph_prod = self.add(ciph_prod, ciph_temp)
        return ciph_prod0
    
    def multiply_matrix(self, ciph, matrix, rot_keys, encoder):
        matrix_len = len(matrix)
        matrix_len_factor1 = int(sqrt(matrix_len))
        if matrix_len != matrix_len_factor1 * matrix_len_factor1:
            matrix_len_factor1 = int(sqrt(3 * matrix_len))
        matrix_len_factor2 = matrix_len // matrix_len_factor1
        ciph_rots = [0] * matrix_len_factor1
        ciph_rots[0] = ciph
        for i in range(1, matrix_len_factor1):
            ciph_rots[i] = self.rotate(ciph, i, rot_keys[i])
        outer_sum = None
        for i in range(matrix_len_factor1):
            diagonal = util.matrix_operations.diagonal(matrix, shift + i)
            diagonal = util.matrix_operations.rotate(diagonal, -shift)
            diagonal_plain = encoder.encode(diagonal, self.scaling_factor)
            dot_prod = self.multiply_plain(ciph_rots[i], diagonal_plain)
            if inner_sum:
                inner_sum = self.add(inner_sum, dot_prod)
            else:
                inner_sum = dot_prod
        rotated_sum = self.rotate(inner_sum, shift, rot_keys[shift])
        if outer_sum:
            outer_sum = self.ass(outer_sum, rotated_sum)
        else:
            outer_sum = rotated_sum
        outer_sum = self.rescale(outer_sum, self.scaling_factor)
        return outer_sum

    def creat_constant_plain(self, const):
        plain_vec = [0] * (self.degree)
        plain_vec[0] = int(const * self.scaling_factor)
        return Plaintext(Polynomial(self.degree, plain_vec, self.scaling_factor))
    
    def creat_complex_constant_plain(self, const, encoder):
        plain_vec = [const] * (self.degree // 2)
        return encoder.encode(plain_vec, self.scaling_factor)
    
    def coeff_to_slot(self, ciph, rot_keys, conj_key, encoder):
        s1 = self.multiply_matrix(ciph, self.boot_context, encoding_mat_conj_transpose0, rot_keys, encoder)
        s2 = self.conjugate(ciph, conj_key)
        s2 = self.multiply_matrix(s2, self.boot_context, encoding_mat_conj_transpose0, rot_keys, encoder)
        ciph0 = self.add(s1, s2)
        constant = self.create_constant_plain(1 / self.degree)
        ciph0 = self.multiply_plain(ciph0, constant)
        ciph0 = self.rescale(ciph0, self.scaling_factor)
        s1 = self.multiply_matrix(ciph, self.boot_context.encoding_mat_conj_transpose1, rot_keys, encoder)
        s2 = self.conjugate(ciph, conj_key)
        s2 = self.multiply_matrix(s2, self.boot_context.encoding_mat_conj_transpose1, rot_keys, encoder)
        ciph1 = self.add(s1, s2)
        ciph1 = self.multiply_plain(ciph1, constant)
        ciph1 = self.rescale(ciph1, self.scaling_factor)
        return ciph0, ciph1
    
    def slot_to_coeff(self, ciph0, ciph1, rot_keys, encoder):
        s1 = self.multiply_matrix(ciph0, self.boot_context.encoding_mat0, rot_keys, encoder)
        s2 = self.multiply_matrix(ciph1, self.boot_context.encoding_mat1, rot_keys, encoder)
        ciph = self.add(s1, s2)
        return ciph
    
    def exp_taylor(self, ciph, relin_key, encoder):
        ciph2 = self.multiply(ciph, ciph, relin_key)
        ciph2 = self.rescale(ciph2, self.scaling_factor)
        ciph4 = self.multiply(ciph2, ciph2, relin_key)
        ciph4 = self.rescale(ciph4, self.scaling_factor)
        
        const = slef.create_constant_plain(1)
        ciph01 = self.add_plain(ciph, const)
        
        const = self.create_constant_plain(1)
        ciph01 = self.multiply_plain(ciph01, const)
        ciph01 = self.rescale(ciph01, self.scaling_factor)
        
        const = self.create_constant_plain(3)
        ciph23 = self.add_plain(ciph, const)
        
        const = self.create_constant_plain(1 / 6)
        ciph23 = self.multiply_plain(ciph23, const)
        ciph23 = self.rescale(ciph23, self.scaling_factor)
        
        ciph23 = self.multiply(ciph23, ciph2, relin_key)
        ciph23 = self.rescale(ciph23, self.scaling_factor)
        ciph01 = self.lower_modulus(ciph01, self.scaling_factor)
        ciph23 = self.add(ciph23, ciph01)
        
        const = self.create_constant_plain(5)
        ciph45 = self.add_plain(ciph, const)
        
        const = self.create_constant_plain(1 / 120)
        ciph45 = self.multiply_plain(ciph45, const) 
        ciph45 = self.rescale(ciph45, self.scaling_factor)
        
        const = self.create_constant_plain(7)
        ciph = self.add_plain(ciph, const)
        
        const = self.creat_complex_constant_plain(1 / 5040)
        ciph = self.multiply_plain(ciph, const)
        ciph = self.rescale(ciph, self.scaling_factor)
        
        ciph45 = self.lower_modulus(ciph45, self.scaling_factor)
        ciph = self.add(ciph, ciph45)
        
        ciph = self.multiply(ciph, ciph4, relin_key)
        ciph = self.rescale(ciph, self.scaling_factor)
        
        ciph23 = self.lower_modulus(ciph23, self.scaling_factor)
        ciph = self.add(ciph, ciph23)
        return ciph
    
    def raise_modulus(self, ciph):
        self.scaling_factor = ciph.modulus
        ciph.scaling_factor = self.scaling_factor
        ciph.modulus = self.big_modulus
    
    def exp(self, ciph, const, relin_key, encoder):
        num_iterations = self.boot_context.num_taylor_iterations
        const_plain = self.creat_complex_constant_plain(const / 2 ** num_iterations, encoder)
        ciph = self.multiply_plain(ciph, const_plain)
        ciph = self.rescale(ciph, self.scaling_factor)
        ciph = self.exp_taylor(ciph, relin_key, encoder)
        for _ in range(num_iterations):
            ciph = self.multiply(ciph, ciph, relin_key)
            ciph = self.rescale(ciph, self.scaling_factor)
        return ciph
    
    def bootstrap(self, ciph, rot_keys, conj_key, relin_key, encoder):
        old_modulus = ciph.modulus 
        old_scaling_factor = self.scaling_factor
        self.raise_modulus(ciph)
        
        ciph0, ciph1 = self.coeff_to_slot(ciph, rot_keys, conj_key, encoder)
        const = self.scaling_factor / old_modulus * 2 * math.pi * 1j
        ciph_exp0 = self.exp(ciph0, const, relin_key, encoder)
        ciph_neg_exp0 = self.conjugate(ciph_exp0, conj_key)
        ciph_exp1 = self.exp(ciph1, const, relin_key, encoder)
        ciph_neg_exp1 = self.conjugate(ciph_exp1, conj_key)
        
        ciph_sin0 = self.subtract(ciph_exp0, ciph_neg_exp0)
        ciph_sin1 = self.subtract(ciph_exp1, ciph_neg_exp1)
        
        

In [3]:
2 * math.pi * 1j

6.283185307179586j

In [ ]:
import time
start = time.time()
a = 1 << 10 ** 10
end = time.time()
print(a, end - start)
print(len(str(a)))

In [27]:
poly_degree = 8
ciph_modulus = 1 << 600
big_modulus = 1 << 1200
scaling_factor = 1 << 30
params = CKKSParametersw(poly_degree = poly_degree,
    ciph_modulus = ciph_modulus,
    big_modulus = big_modulus,
    scaling_factor = scaling_factor)

print(params)

86 <util.crt.CRTContext object at 0x7f72bca875d0>


In [5]:
CKKEKeyGenerator.generate_secret_key(1,10)

AttributeError: 'int' object has no attribute 'poly_degree'